In [ ]:
from lambeq import BobcatParser, stairs_reader, cups_reader
from lambeq import AtomicType, IQPAnsatz
from quantum_pipeline import *
from utilities import *

In [2]:
N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

ansatz = IQPAnsatz({N: 1, S: 1, C: 1, P: 1, NP: 1, PP: 1}, n_layers = 1, n_single_qubit_params = 3)

In [3]:
pipeline = QuantumPipeline(stairs_reader, ansatz)
#gps_labels, gps_circuits = pipeline.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/quantum/GPS_quantum.csv")
#cpn_labels, cpn_circuits = pipeline.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/quantum/CPN_quantum.csv")
#epurse_labels, epurse_circuits = pipeline.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/quantum/ePurse_quantum.csv")

In [4]:
#save_data("quantum_gps.txt", gps_labels, gps_circuits)
#save_data("quantum_cpn.txt", cpn_labels, cpn_circuits)
#save_data("quantum_epurse.txt", epurse_labels, epurse_circuits)

gps_labels, gps_circuits = load_data("quantum_gps.txt")
cpn_labels, cpn_circuits = load_data("quantum_cpn.txt")
epurse_labels, epurse_circuits = load_data("quantum_epurse.txt")

In [ ]:
from sklearn.model_selection import KFold

circ = gps_circuits + cpn_circuits + epurse_circuits
lab = gps_labels + cpn_labels + epurse_labels

mean_acc = 0
mean_prec = 0
mean_rec = 0
mean_f1 = 0

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
for i, (train_index, test_index) in enumerate(kf.split(circ)):
    print(f"Fold {i+1}:")
    x_train = [circ[index] for index in train_index]
    x_test =[circ[index] for index in test_index]
    y_train = [lab[index] for index in train_index]
    y_test = [lab[index] for index in test_index]

    model = pipeline.create_model(circ)
    train_set = pipeline.create_dataset(x_train, y_train)
    pipeline.create_trainer(model = model, n_epochs = 100, a_hyp = 3e-2, evaluate = False)
    pipeline.train_model(train_set, 10, 10)
    
    
    test_accuracy = accuracy(model(x_test), torch.tensor(y_test))
    print(f"Test accuracy on fold {i+1}: {test_accuracy.item()}")
    mean_acc += test_accuracy
    
    test_precision = precision(model(x_test), torch.tensor(y_test))
    print(f"Test precision on fold {i+1}: {test_precision.item()}")
    mean_prec += test_precision
    
    test_recall = recall(model(x_test), torch.tensor(y_test))
    print(f"Test recall on fold {i+1}: {test_recall.item()}")
    mean_rec += test_recall
    
    test_f1 = f1score(model(x_test), torch.tensor(y_test))
    print(f"Test f1_score on fold {i+1}: {test_f1.item()}")
    mean_f1 += test_f1
    
mean_acc /= 5
mean_prec /= 5
mean_rec /= 5
mean_f1 /= 5

print(f"Metrics mean:\n\t- accuracy: {mean_acc}\n\t- precision: {mean_prec}\n\t- recall: {mean_rec}\n\t- f1: {mean_f1}\n\n")